## Analytics Metro-Connexion

### Import Libraries

In [ ]:
import polars as pl
from dotenv import load_dotenv, dotenv_values
from pathlib import Path
import os
import json 
from openai import OpenAI
import pandas as pd

### Import Metro Connection data

In [ ]:
def read_metro_conexion():
    df = pl.read_csv("/home/onyxia/work/hackathon_mobilites_2025/data/raw/metro_connexion_corresp_idfm_ref.csv")
    return df

df = read_metro_conexion()
df.head()

/home/onyxia/work/hackathon_mobilites_2025/notebooks


href,Provenance,Destination,Station,Description détaillée,Ligne de Provenance,Ligne de Destination,ID Zone arret ICAR,ID Ligne de Destination,ID Ligne de Provenance
str,str,str,str,str,str,str,i64,str,str
"""https://www.metro-connexion.or…","""Champ de Mars, Pontoise, Versa…","""Mairie de Montreuil""","""Alma - Marceau Pont de l'Alma""","""Via la station Alma-Marceau. …","""C""","""9""",42182,"""C01379""","""C01727"""
"""https://www.metro-connexion.or…","""Champ de Mars, Pontoise, Versa…","""Champ de Mars, Pontoise, Versa…","""Alma - Marceau Pont de l'Alma""","""La correspondance pour retourn…","""C""","""C""",42182,"""C01727""","""C01727"""
"""https://www.metro-connexion.or…","""Champ de Mars, Pontoise, Versa…","""Pont de Sèvres""","""Alma - Marceau Pont de l'Alma""","""Via la station Alma-Marceau. …","""C""","""9""",42182,"""C01379""","""C01727"""
"""https://www.metro-connexion.or…","""Choisy-le-Roi, Massy-Palaiseau…","""Mairie de Montreuil""","""Alma - Marceau Pont de l'Alma""","""Via la station Alma-Marceau. …","""C""","""9""",42182,"""C01379""","""C01727"""
"""https://www.metro-connexion.or…","""Choisy-le-Roi, Massy-Palaiseau…","""Choisy-le-Roi, Massy-Palaiseau…","""Alma - Marceau Pont de l'Alma""","""La correspondance pour retourn…","""C""","""C""",42182,"""C01727""","""C01727"""


### Select an example of the description data

In [13]:
sample_txt = df.to_pandas().iloc[:1,4].values[0]

### Write an LLM col, to extract quantitative indicators from the data

In [ ]:
def answer_llm(text: str):
    #Use the LLM provided for the Hackathon (we chose gpt-4o-mini)
    path_env = Path(os.getcwd()) / ".." / ".env"
    config = dotenv_values(path_env)

    ed_team = config["ENDPOINT"]
    api_key = config["APIKEY"]

    endpoint= f"{ed_team}" + "openai/v1/"

    model_name = "gpt-4o-mini"

    client = OpenAI(
        base_url=f"{endpoint}",
        api_key=api_key
    )

    # Instruction système pour garantir un format JSON strict et une analyse précise
    system_instruction = (
        "You are an expert data extractor specialized in extracting travel data for accessibility reports. "
        "Analyze the provided French text which describes a walking itinerary. "
        "Your task is to extract three specific pieces of numerical information, counting based on the textual mentions: "
        "1. The total number of *explicitly mentioned individual steps* ('marches' or 'marche') in the entire text. (e.g., 'quatre marches' = 4, 'une seule marche' = 1). Sum these explicit counts. Ignore 'escalier' mentions unless an explicit number of steps is given."
        "2. The total distance traveled in meters. (Sum all explicit meter values, e.g., '10 mètres' = 10, 'une centaine de mètres' = 100). Ignore vague mentions like 'quelques mètres'."
        "3. The total count of ascent and descent actions or structures (escaliers montants/descendants, monter/descendre). "
        "Provide the result as a single JSON object with the following keys and integer values: `total_stairs`, `total_meters`, `ascendings`, `descendings`."
    )
    
    
    # Tentative d'appel à l'API avec le format de réponse JSON
    try:
        response = client.chat.completions.create(
            model=model_name,
            messages=[
                {"role": "system", "content": system_instruction},
                {"role": "user", "content": f"Analyze the following itinerary text and provide the structured output:\n\n---\n{sample_txt}"}
            ],
            # Ceci est la clé pour forcer une réponse JSON structurée
            response_format={"type": "json_object"}
        )

        # Le contenu est retourné sous forme de chaîne JSON
        json_string = response.choices[0].message.content
        
        # Afficher la réponse brute et le JSON analysé
        #print("--- Réponse JSON brute de l'API ---")
        # print(json_string)

        # Analyser la chaîne JSON pour l'utiliser en Python
        structured_data = json.loads(json_string)
        
        # print("\n--- Données structurées extraites ---")
        # print(f"Nombre total de marches explicitement mentionnées : {structured_data.get('total_stairs', 'N/A')}")
        # print(f"Distance totale parcourue (mètres) : {structured_data.get('total_meters', 'N/A')}")
        # print(f"Nombre de montées (escaliers/marches) : {structured_data.get('ascendings', 'N/A')}")
        # print(f"Nombre de descentes (escaliers/marches) : {structured_data.get('descendings', 'N/A')}")

        return structured_data

    except Exception as e:
        print(f"Une erreur est survenue lors de l'appel à l'API : {e}")

# Call the LLM for just the example row
answer_llm(sample_txt)

--- Réponse JSON brute de l'API ---


{'total_stairs': 12, 'total_meters': 125, 'ascendings': 5, 'descendings': 5}

### Convert dataframe into Pandas And call the LLM on all rows (write result in col "extraction")

In [33]:
df_pd = df.to_pandas()

df_pd['Extraction'] = df_pd['Description détaillée'].map(answer_llm)

--- Réponse JSON brute de l'API ---
--- Réponse JSON brute de l'API ---
--- Réponse JSON brute de l'API ---
--- Réponse JSON brute de l'API ---
--- Réponse JSON brute de l'API ---
--- Réponse JSON brute de l'API ---
--- Réponse JSON brute de l'API ---
--- Réponse JSON brute de l'API ---
--- Réponse JSON brute de l'API ---
--- Réponse JSON brute de l'API ---
--- Réponse JSON brute de l'API ---
--- Réponse JSON brute de l'API ---
--- Réponse JSON brute de l'API ---
--- Réponse JSON brute de l'API ---
--- Réponse JSON brute de l'API ---
--- Réponse JSON brute de l'API ---
--- Réponse JSON brute de l'API ---
--- Réponse JSON brute de l'API ---
--- Réponse JSON brute de l'API ---
--- Réponse JSON brute de l'API ---
--- Réponse JSON brute de l'API ---
--- Réponse JSON brute de l'API ---
--- Réponse JSON brute de l'API ---
--- Réponse JSON brute de l'API ---
--- Réponse JSON brute de l'API ---
--- Réponse JSON brute de l'API ---
--- Réponse JSON brute de l'API ---
--- Réponse JSON brute de l'

In [ ]:
#Format the extraction col correctly
df_merged_qualif = pd.merge(df_pd, pd.json_normalize(df_pd['Extraction']), left_index=True, right_index=True)
df_merged_qualif.head(2)

,href,Provenance,Destination,Station,Description détaillée,Ligne de Provenance,Ligne de Destination,ID Zone arret ICAR,ID Ligne de Destination,ID Ligne de Provenance,Extraction,total_stairs,total_meters,ascendings,descendings
0,https://www.metro-connexion.org/index.php?modu...,"Champ de Mars, Pontoise, Versailles-rive gauch...",Mairie de Montreuil,Alma - Marceau Pont de l'Alma,Via la station Alma-Marceau.\r\nLa corresponda...,C,9,42182,C01379,C01727,"{'total_stairs': 11, 'total_meters': 164, 'asc...",11,164,6,5
1,https://www.metro-connexion.org/index.php?modu...,"Champ de Mars, Pontoise, Versailles-rive gauch...","Champ de Mars, Pontoise, Versailles-rive gauch...",Alma - Marceau Pont de l'Alma,La correspondance pour retourner vers Champ-de...,C,C,42182,C01727,C01727,"{'total_stairs': 17, 'total_meters': 192, 'asc...",17,192,5,5


### Write Dataframe into a parquet file

In [ ]:
df_merged_qualif.to_parquet("../data/interim/metro_conexion_qualif.parquet")